# Building a Project Manager (PM) Agent for VM Migration in Python

## What You'll Learn

In this notebook, you will learn how to build and deploy a Project Manager (PM) agent designed to manage the execution of a Virtual Machine (VM) migration plan. This agent will coordinate various tasks, monitor progress, and dynamically adapt the plan based on feedback from different engineering agents. By the end of this notebook, you will understand how to:

- Set up and configure a model to support the PM agent's decision-making process.
- Define and manage the state for the PM agent to track task progress and dependencies.
- Implement and customize the PM agent to handle specific responsibilities within the VM migration workflow.
- Construct and compile a workflow graph to represent the PM agent’s interactions with other agents.
- Execute the workflow, manage outputs, and handle dynamic adjustments to the plan based on real-time feedback.

## Basic Concepts

Before diving into the implementation, it's essential to understand some basic concepts:

- **PM Agent:** A specialized agent that autonomously manages the execution of a VM migration plan. The PM agent coordinates tasks, assigns responsibilities to other agents (e.g., OCP Engineer, vSphere Engineer), and tracks the overall progress of the migration.

- **State:** A shared data structure that stores the context, task statuses, dependencies, and other critical information required by the PM agent. Effective state management ensures that the PM agent can track progress, handle feedback, and adapt the plan as needed.

- **VM Migration Plan:** A structured set of tasks required to move virtual machines from one environment to another. The PM agent will break down the migration plan into individual tasks, assign them to the appropriate agents, and ensure they are executed in the correct order.

- **Workflow Graph:** A structured representation of the PM agent's workflow, where nodes represent tasks or decisions, and edges define the flow of control between these steps. This graph helps visualize and manage the sequence of actions during the VM migration process.

- **Feedback Handling:** The process by which the PM agent receives and processes feedback from other agents (e.g., task completion, errors, or issues) and adapts the migration plan accordingly to ensure a smooth execution.

- **Prompt Engineering:** The process of crafting prompts that guide the PM agent's interaction with the model. Proper prompt engineering ensures that the agent generates relevant and accurate tasks and adjustments to the migration plan.

Understanding these concepts will provide a solid foundation as we proceed with the practical implementation of the PM agent and its role in managing a VM migration plan.


## 1. Setup

Before we begin, let's make sure your environment is set up correctly. We'll start by installing the necessary Python packages.

### Installing Required Packages

To get started, you'll need to install a few Python libraries. Run the following command to install them:

In [19]:
%pip install langchain langgraph langgraph-checkpoint-sqlite requests termcolor

Note: you may need to restart the kernel to use updated packages.


## 2. Building Blocks

In this section, we'll start by building the basic components that our agent will use. These building blocks will form the foundation of our agent, enabling it to keep track of time, store data, and interact with a model.

### Datetime Function

The first building block we'll create is a simple function to get the current time. This is important because our agent might need to timestamp certain actions or events. Let's write a function that returns the current date and time in UTC format:


In [20]:
from datetime import datetime, timezone


def get_current_utc_datetime():
    now_utc = datetime.now(timezone.utc)
    return now_utc.strftime("%Y-%m-%d %H:%M:%S.%f UTC")[:-3]


# Example usage:
print("Current UTC datetime:", get_current_utc_datetime())

Current UTC datetime: 2024-09-04 20:16:56.436355 


### Basic Agent Data Handling
Next, we'll introduce a simple way to handle data for our agent. In this case, we'll create a function that allows us to load and manage the agent's data. This data might include things like the agent's responsibilities, the tasks it needs to perform, and other relevant information.

Let's assume the data is stored in a YAML file (a common format for configuration files), and we'll write a function to load this data:

In [21]:
import yaml
from typing import Dict, Any


def load_agent_descriptions(description_file: str) -> Dict[str, Any]:
    """
    Load the agent descriptions from a YAML file.
    """
    try:
        with open(description_file, "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"Description file '{description_file}' not found.")


# Example usage:
agents_description = load_agent_descriptions("agents.yaml")
print("Loaded agent descriptions:", agents_description)

Loaded agent descriptions: {'agents': [{'name': 'Planner Agent', 'role': 'Planner Agent', 'responsibilities': ['Creates a comprehensive Migration Plan based on the tutorial.', 'Identifies key steps, target VMs, and source/target providers.', 'Coordinates and structures the plan for execution by other agents.']}, {'name': 'Project Manager (PM) Agent', 'role': 'Project Manager Agent', 'responsibilities': ['Manages the breakdown of tasks for the migration process.', 'Oversees task execution and ensures agents are working in coordination.', 'Ensures timelines are followed and adjusts the plan as necessary.', 'Communicates with all agents to ensure smooth task progression and resolve bottlenecks.']}, {'name': 'vSphere Engineer Agent', 'role': 'vSphere Engineer Agent', 'responsibilities': ['Handles VM identification and configuration within the vSphere environment.', 'Identifies VMs to migrate based on the tutorial instructions.', 'Manages and configures the Migration Toolkit for Virtualizat

## 3. Configuring a Simple Model
Now that we have the basic building blocks, we'll move on to configuring a model that our agent can use to perform its tasks. This model will process inputs (like a user request) and generate outputs (like a task list).

### Model Configuration
We'll start by setting up a simple configuration for the model. This configuration will include details like the model's endpoint, temperature, and other parameters. Let's create a function to handle this:

In [22]:
def setup_ollama_model(
    model, temperature=0.0, top_p=1.0, top_k=0, repetition_penalty=1.0, stop=None
):
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "repetition_penalty": repetition_penalty,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration:
ollama_config = setup_ollama_model(model="llama3:instruct")
print("Model configuration:", ollama_config)

Model configuration: {'model_endpoint': 'http://localhost:11434/api/generate', 'model': 'llama3:instruct', 'temperature': 0.0, 'top_p': 1.0, 'top_k': 0, 'repetition_penalty': 1.0, 'headers': {'Content-Type': 'application/json'}, 'stop': None}


This function returns a dictionary with the model's configuration. You can adjust the parameters based on the specific model you're using or the task requirements.

### Preparing a Request
With the model configured, the next step is to prepare a request that the agent can send to the model. This request will include the user's input, the system's instructions, and any other necessary information. Let's write a function to prepare this request:

In [23]:
def prepare_payload(
    user_prompt: str,
    sys_prompt: str,
    stream: bool = False,
    config: Dict[str, Any] = ollama_config,
) -> Dict[str, Any]:
    return {
        "model": config.get("model"),
        "format": "json",
        "prompt": user_prompt,
        "system": sys_prompt,
        "stream": stream,
        "temperature": config.get("temperature", 0.0),
        "top_p": config.get("top_p", 1.0),
        "top_k": config.get("top_k", 0),
        "repetition_penalty": config.get("repetition_penalty", 1.0),
        "stop": config.get("stop"),
    }

### Sending a Request
We'll start by writing a function to send the request to the model's endpoint and receive a response:

In [24]:
import requests
import json


def request_model_generate_endpoint(
    payload: Dict[str, Any], config: Dict[str, Any] = ollama_config
) -> Dict[str, Any]:
    try:
        response = requests.post(
            config.get("model_endpoint"),
            headers=config.get("headers", {"Content-Type": "application/json"}),
            data=json.dumps(payload),
            timeout=30,
        )
        response.raise_for_status()

        if response.content.strip():
            return response.json()
        else:
            return {"error": "Empty response from model"}

    except requests.RequestException as e:
        raise Exception(f"Request failed: {e}")

This function sends the prepared payload to the model's endpoint using the `requests` library. It then checks if the response is valid and returns the content. If there's an error in the request, it raises an exception with a descriptive message.

### Processing the Response
Finally, we'll write a function to process and understand the model's response. This might involve formatting the response or extracting specific information:

In [25]:
def process_model_response(response_json: Dict[str, Any]) -> str:
    try:
        response_content = json.loads(response_json.get("response", "{}"))
        pretty_content = json.dumps(response_content, indent=4)

        return pretty_content
    except json.JSONDecodeError:
        return "Error processing the response"

## 4. Custom Tools

In [26]:
import operator
from langchain.tools import tool


@tool(parse_docstring=True)
def basic_calculator(num1, num2, operation):
    """
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    'num1' (int): The first number.
    'num2' (int): The second number.
    'operation' (str): The operation to perform. Supported operations are 'add', 'subtract',
                        'multiply', 'divide', 'floor_divide', 'modulus', 'power', 'lt',
                        'le', 'eq', 'ne', 'ge', 'gt'.

    Returns:
    str: The formatted result of the operation.

    Raises:
    Exception: If an error occurs during the operation (e.g., division by zero).
    ValueError: If an unsupported operation is requested or input is invalid.
    """

    # Define the supported operations
    operations = {
        "add": operator.add,
        "subtract": operator.sub,
        "multiply": operator.mul,
        "divide": operator.truediv,
        "floor_divide": operator.floordiv,
        "modulus": operator.mod,
        "power": operator.pow,
        "lt": operator.lt,
        "le": operator.le,
        "eq": operator.eq,
        "ne": operator.ne,
        "ge": operator.ge,
        "gt": operator.gt,
    }

    # Check if the operation is supported
    if operation in operations:
        try:
            # Perform the operation
            result = operations[operation](num1, num2)
            result_formatted = (
                f"\n\nThe answer is: {result}.\nCalculated with basic_calculator."
            )
            return result_formatted
        except Exception as e:
            return str(e), "\n\nError during operation execution."
    else:
        return "\n\nUnsupported operation. Please provide a valid operation."

In [27]:
from langchain.tools.render import render_text_description_and_args

# To use these tools within our agent, we register them in a list. This list will be referenced by the agent to determine which tools are available for use.
tools = [basic_calculator]
tools_description = (
    render_text_description_and_args(tools).replace("{", "{{").replace("}", "}}")
)

## 5. State Management

State management is an essential concept when working with agents. The "state" of an agent refers to its current condition or the information it has at any given time. For instance, if an agent is working through a list of tasks, its state might include which tasks have been completed, which are in progress, and which are yet to be started.

Managing this state is crucial because it allows the agent to keep track of what it has done and what it needs to do next. Without proper state management, an agent might lose track of its progress, repeat tasks, or skip important steps. In this notebook, you'll learn how to manage an agent's state effectively, ensuring that it operates smoothly and efficiently.

### Implementation of State Management

To implement state management for our PM agent, we define a structured data model (`AgentGraphState`) that holds all the relevant information the agent needs to function effectively. This model includes:
- **Input:** The current command or task that the agent is working on.
- **Response:** The outputs or actions the agent has generated.
- **Feedback:** Any feedback received from other agents or parts of the system.
- **Validation Status:** A flag indicating whether the agent's output has been validated, ensuring that all tasks meet the required criteria before proceeding.

Additionally, we provide a utility function, `update_state`, which allows for updating specific elements of the agent's state. This function ensures that the state is consistently and accurately maintained, which is critical for the agent to operate effectively. By checking for the existence of keys before updating, the function helps prevent errors and maintains the integrity of the state.

Together, these components form the backbone of the agent's state management system, enabling it to manage complex workflows and adapt to changes dynamically.

In [28]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any

class AgentGraphState(TypedDict):
    input: str
    response: Annotated[list, add_messages]
    feedback: Annotated[list, add_messages]
    validated: bool = False


def update_state(state: AgentGraphState, key: str, value: Any):
    """
    Update the state of the agent. Warn if the key doesn't exist.
    """
    if key in state:
        state[key] = value
    else:
        print(f"Warning: Attempting to update a non-existing state key '{key}'.")

## 6. What is an Agent?

In simple terms, an agent is a program that can perform tasks autonomously based on a set of instructions. Think of an agent as a virtual assistant that can handle specific jobs for you. For example, in the context of data science or software development, an agent might process data, make decisions based on that data, and then carry out actions like sending requests or updating records.

Agents are often designed to work without constant human intervention. Once you give them the initial instructions, they can execute tasks on their own, making them very useful in automating repetitive or complex processes.


### System Prompts

System prompts are predefined instructions provided to large language models (LLMs) that guide the AI's behavior during interactions. They set the context, define the role of the AI, and establish rules for how the model should respond to user inputs.

#### Importance of System Prompts

System prompts are essential for ensuring that an AI model like our PM agent performs its tasks consistently and effectively. By specifying the role, task guidelines, and response format, the system prompt helps the AI maintain focus on its objectives—in this case, managing a VM migration plan. 

#### How They Work

When the PM agent receives a query, the system prompt shapes how it processes the input and generates its output. This includes breaking down tasks, assigning them to the correct agents, and handling feedback—all within the structure provided by the prompt.

In summary, system prompts are a powerful tool that directs AI behavior, ensuring responses are aligned with the desired goals and context.


In [29]:
DEFAULT_SYS_REACT_PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython  
Knowledge Cutoff Date: December 2023  
Current Date: {datetime}

You are an intelligent assistant designed to handle various tasks, including answering questions, providing summaries, and performing detailed analyses. All outputs must strictly be in JSON format.

---

## Tools
You have access to a variety of tools to assist in completing tasks. You are responsible for determining the appropriate sequence of tool usage to break down complex tasks into subtasks when necessary.

The available tools include:

{tools_description}

---

## Output Format:
To complete the task, please use the following format:

{{
  "thought": "Describe your thought process here, including why a tool may be necessary to proceed.",
  "action": "Specify the tool you want to use.",
  "action_input": {{ # Provide valid JSON input for the action, ensuring it matches the tool’s expected format and data types.
    "key": "Value inputs to the tool in valid JSON format."
  }}
}}

After performing an action, the tool will provide a response in the following format:

{{
  "observation": "The result of the tool invocation",
}}

You should keep repeating the format (thought → action → observation) until you have gathered enough information to answer the question. **If the observation provides a clear and complete answer to the user's query, immediately conclude with the final answer and do not perform further actions.** Once you have sufficient information, respond using one of the following formats:


If the tool result is successful and the task is complete:

{{
  "thought": "The tool '{{action}}' executed successfully, and the output meets the acceptance criteria. No further actions are required.",
  "final_answer": "The task has been completed successfully with the tool output: {{tool_result}}."
}}

Or, if you cannot answer:

{{
  "thought": "The tool '{{action}}' failed to execute successfully. The error was: {{tool_result}}. Here is what went wrong and what needs to be corrected: [Provide corrections or adjustments].",
  "action_correction": "Description of what needs to be adjusted or corrected before retrying."
}}

---

### Remember:
- Use the tools effectively and ensure inputs match the required format exactly as described in the task.
- **If a tool provides a complete and clear answer, do not continue invoking further tools.**
- Maintain the JSON format and ensure all fields are filled out correctly.
- Do not include additional metadata such as `title`, `description`, or `type` in the `tool_input`.

---

## Current Conversation
Below is the ongoing conversation consisting of interleaving human and assistant messages:

{agent_scratchpad}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
"""

In [30]:
def write_react_prompt(
    agent_scratchpad: str = "",
    tools_description: str = tools_description,
) -> str:
    return DEFAULT_SYS_REACT_PROMPT.format(
        agent_scratchpad=agent_scratchpad,
        tools_description=tools_description,
        datetime=get_current_utc_datetime(),
    )

In [31]:
# Example usage:
payload = prepare_payload(
    user_prompt="What tasks should I complete in order to make pasta?",
    sys_prompt=write_react_prompt(),
)
print("Prepared payload:", payload)

Prepared payload: {'model': 'llama3:instruct', 'format': 'json', 'prompt': 'What tasks should I complete in order to make pasta?', 'system': '\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nEnvironment: ipython  \nKnowledge Cutoff Date: December 2023  \nCurrent Date: 2024-09-04 20:16:56.538407 \n\nYou are an intelligent assistant designed to handle various tasks, including answering questions, providing summaries, and performing detailed analyses. All outputs must strictly be in JSON format.\n\n---\n\n## Tools\nYou have access to a variety of tools to assist in completing tasks. You are responsible for determining the appropriate sequence of tool usage to break down complex tasks into subtasks when necessary.\n\nThe available tools include:\n\nbasic_calculator(num1, num2, operation) - Perform a numeric operation on two numbers based on the input string. Parameters:\n\'num1\' (int): The first number.\n\'num2\' (int): The second number.\n\'operation\' (str): The operatio

## Step 7: Implementing the Agent

### Creating a Simple Agent Class

In this step, we define the `Agent` class, which is responsible for managing the state of the PM agent, interacting with the model, and processing the responses. This class encapsulates the core functionalities needed to execute tasks autonomously.

### Key Components:
- **Initialization (`__init__`)**: The constructor initializes the agent with its state, role, and model configuration. This setup is crucial for ensuring that the agent operates within the defined parameters and context.
  
- **Model Invocation (`invoke_model`)**: This method prepares the input payload, sends it to the model for processing, and handles the model's response. It's where the agent interacts with the LLM, using the system prompt and user prompt to generate meaningful outputs.

- **Task Execution (`execute_task`)**: This method allows the agent to execute a specific task based on a user request. It utilizes the system prompt tailored for the task and processes the response generated by the model.

The `Agent` class is fundamental in making our PM agent autonomous, enabling it to perform its duties without constant human oversight.

In [32]:
from langchain_core.messages.ai import AIMessage
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from termcolor import colored


class Agent:
    def __init__(self, state: AgentGraphState, role: str, model_config: dict):
        """
        Initialize the Agent with a state, role, and model configuration.
        """
        self.state = state
        self.role = role
        self.model_config = model_config

    def invoke_model(self, sys_prompt: str, user_prompt: str):
        """
        Prepare the payload, send the request to the model, and process the response.
        """
        # Prepare the payload
        payload = prepare_payload(user_prompt, sys_prompt, config=self.model_config)

        # Invoke the model and get the response
        response_json = request_model_generate_endpoint(
            payload, config=self.model_config
        )

        # Process the model's response
        response_content = process_model_response(response_json)

        # Return the processed response
        return response_content

    def react(self, user_request: str) -> dict:
        """
        Execute the task based on the user's request by following the thought → action → observation loop.
        """
        sys_prompt = write_react_prompt()
        final_answer = None

        # Start with the user's request as the first input
        user_prompt = user_request
        action = None
        action_input = None
        scratchpad = []

        human_message = HumanMessage(content=user_prompt)
        print(colored(human_message.pretty_repr(), "green"))

        # Loop until a final answer is generated
        while final_answer is None:
            # Invoke the model with the system prompt and current user input

            response = self.invoke_model(sys_prompt=sys_prompt, user_prompt=user_prompt)

            try:
                # Parse the response assuming it's in JSON format
                response_dict = json.loads(response)  # Assuming response is a JSON object

                ai_message = AIMessage(content=response)
                print(colored(ai_message.pretty_repr(), "cyan"))

                scratchpad.append(ai_message)

                action = response_dict.get("action", None)
                action_input = response_dict.get("action_input", None)

                # If there is an action, execute the corresponding tool
                if action and action_input:
                    status, tool_response = self.execute_tool(action, action_input)

                    # Formulate the observation to feed back into the model
                    tool_response_dict = {
                        "observation": tool_response,
                    }

                    tool_response_json = json.dumps(tool_response_dict, indent=4)

                    tool_system_message = SystemMessage(content=tool_response_json)
                    print(colored(tool_system_message.pretty_repr(), "yellow"))

                    user_prompt = tool_response_json

                # Check if the model has given a final answer
                if "final_answer" in response_dict:
                    final_answer = response_dict["final_answer"]

            except Exception as e:
                print(str(e))
                error_message = SystemMessage(content=str(e))
                scratchpad.append(error_message)

        # Return the final answer
        return {"response": AIMessage(content=final_answer)}

    def execute_tool(self, action: str, action_input: dict):
        """
        Simulate the tool execution based on the action and action_input.
        In a real-world scenario, this would call the appropriate tool.
        """
        # Simulate some tool actions (this would be replaced by actual tool logic)
        print(
            colored(
                "================================ Calling Tool ================================",
                "magenta",
            )
        )

        print(colored(f"Tool: {action}", "magenta"))
        print(colored(f"Tool Input: {action_input}", "magenta"))

        for tool in tools:
            if tool.name == action:
                try:
                    result = tool.invoke(action_input)
                    print(colored(f"Tool Result: {result}", "magenta"))
                    return True, result
                except Exception as e:
                    return False, f"Error executing tool {action}: {str(e)}"
        else:
            return f"Tool {action} not found or unsupported operation."

## Step 8: Creating and Compiling the Workflow Graph

### Constructing the PM Agent's Workflow

In this step, we build the workflow graph that represents the PM agent's process. This graph outlines the sequence of operations, including task execution, validation, and decision-making.

### Key Components:
- **Node Definitions**: Each node in the graph represents a step in the workflow, such as invoking the PM agent or validating the output.
  
- **Edge Definitions**: Edges define the flow between nodes, determining how the agent progresses through the tasks and validation steps.

- **Workflow Compilation**: Once the graph is defined, it is compiled into a workflow that can be executed. This compiled workflow represents the full sequence of operations that the PM agent will follow to manage the VM migration.

By constructing and compiling this workflow graph, we ensure that the PM agent operates in a structured and efficient manner, handling tasks and making decisions in a logical sequence.

In [33]:
def react_node_function(state: AgentGraphState):
    react_agent = Agent(
        state=state,
        role="REACT_AGENT",
        model_config=ollama_config,
    )

    return react_agent.react(user_request=state["input"])

In [34]:
from langgraph.graph import StateGraph, START, END


def create_graph() -> StateGraph:
    """
    Create the state graph by defining nodes and edges.

    Returns:
    - StateGraph: The compiled state graph ready for execution.
    """
    graph = StateGraph(AgentGraphState)

    # Add nodes
    graph.add_node("react_agent", react_node_function)

    # Define the flow of the graph
    graph.add_edge(START, "react_agent")
    graph.add_edge("react_agent", END)

    return graph

### Initializing Sqlite Persistence for Graph State

In this cell, we define and use a method to initialize an `SqliteSaver` instance from the `langgraph.checkpoint.sqlite` module. The `SqliteSaver` class allows the graph state to be persisted in an SQLite database, which is more durable and suitable for applications requiring longer-term storage compared to an in-memory solution.

The `from_conn_stringx` method is defined as a class method that takes a connection string as input, creates a connection to the SQLite database using `sqlite3.connect`, and then returns an `SqliteSaver` instance using this connection. This method simplifies the creation of an `SqliteSaver` instance directly from a connection string.

This approach is particularly useful for ensuring that the state of the `StateGraph` is saved to a local or memory-based SQLite database, enabling the retention of context across multiple interactions in AI-driven applications.

In [35]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3


def from_conn_stringx(
    cls,
    conn_string: str,
) -> "SqliteSaver":
    return SqliteSaver(conn=sqlite3.connect(conn_string, check_same_thread=False))


SqliteSaver.from_conn_stringx = classmethod(from_conn_stringx)

memory = SqliteSaver.from_conn_stringx(":memory:")

## Step 9: Executing the Workflow

With the workflow graph compiled, the final step is to execute the workflow. This involves providing the agent with an input query, such as a request to generate a VM migration plan, and allowing the workflow to run through its defined sequence.

### Creating and Running the Workflow

In this step, we create and execute the PM agent's workflow to process a set of tasks.

- **Graph Creation**: 
  - We first create the workflow graph using `create_graph()` and compile it with a memory-based checkpoint.
  - The compiled workflow will manage the task execution, validation, and feedback handling.

- **Workflow Parameters**:
  - We define the number of iterations (`iterations = 10`), set verbose mode to `True`, and configure the thread ID.
  - A query containing three tasks (VM details retrieval, migration plan creation, and migration start) is provided as input.

- **Workflow Execution**:
  - The workflow is executed using `workflow.stream()`, and it processes each task sequentially.
  - Depending on the state of the workflow, feedback or task responses are printed to track progress.

This step runs the agent through the defined tasks and prints the state changes for each event in the workflow.

In [36]:
# Create the graph and compile the workflow
graph = create_graph()
workflow = graph.compile(checkpointer=memory)
print("Graph and workflow created.")

# Define workflow parameters
iterations = 10
verbose = True
config = {"configurable": {"thread_id": "1"}}

query = "What is 10+10?"
dict_inputs = {"input": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
            print("\nEvent:", event)
    else:
        print("\n")

Graph and workflow created.
================================ Human Message =================================

What is 10+10?
================================== Ai Message ==================================

{
    "thought": "To determine the result, I will use the basic calculator tool to add two numbers.",
    "action": "basic_calculator",
    "action_input": {
        "num1": 10,
        "num2": 10,
        "operation": "add"
    }
}
================================ Calling Tool ================================
Tool: basic_calculator
Tool Input: {'num1': 10, 'num2': 10, 'operation': 'add'}
Tool Result: 

The answer is: 20.
Calculated with basic_calculator.
================================ System Message ================================

{
    "observation": "\n\nThe answer is: 20.\nCalculated with basic_calculator."
}
================================== Ai Message ==================================

{
    "thought": "It seems like we've already calculated the result, so there's no nee